<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

**Описание**

В нашем распоряжении набор данных с разметкой о токсичности комментариев. Стоит задача обучить модель классифицировать комментарии на позитивные и негативные. 

- text - комментарий пользователя
- toxic - является ли комментарий токсичным или нет (1 или 0)

**Задачи**

- исследовать и подготовить данные
- обучить разные модели для классификации комментариев на позитивные и негативные.
- добиться значения метрики качества F1 не меньше 0.75
- сделать вывод

In [1]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
try: 
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df.sample(10)

,Unnamed: 0,text,toxic
108718,108815,"Another thing, PartyPoker the international ga...",0
107728,107825,"""\n\n Rotten Tomatoes \n\nThe article says it ...",0
83505,83584,Blocking Anthony Robbins \n\nYou fucking moron...,1
154612,154769,"""\n\nhehe, you *ARE* truly pathetic Zereshk, a...",1
7420,7432,I can live with this. I like reliably sourced...,0
86033,86114,Really? I think you did.,0
80667,80743,Removed Redirect\nI removed the redirect of th...,0
115135,115234,Thanks for your note. Can you back that up wit...,0
98389,98484,Speechless \n\nIs that a good speechless or a ...,0
151267,151423,"I understand. I'll leave the wording as it is,...",0


Датасет содержит 159292 строки, типы данных ок

удалим дублирующий столбец

In [7]:
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
df.shape

(159292, 2)

In [9]:
df.isna().sum() 

text     0
toxic    0
dtype: int64

пропусков нет

In [10]:
df.duplicated().sum()

0

дубликатов нет

В датасете 159292 строк и 2 столбца. Пропущенных значений и дубликатов нет. Целевым признаком является `toxic` -оценка токсичности в комментарии `text`

Посмотрим сколько у нас токсичных/нектоксичных текстов. (распределение целевого признака)

In [11]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [12]:
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

8.841344371679229

На лицо сильный дисбаланс. Позитивных комментариев сильно больше, чем токсичных. обучим модель с выставлением параметра `class_weight='balanced'`

Комментарии выгружены с какими-то специальными символами и числами. Очистим их с помощью регулярных выражений

In [13]:
def clear_text(text):
    text = re.sub(r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", ' ', text.lower()) # преобразовываем и переводим в нижний регистр
    retext = text.split() 
    text = " ".join(retext)
    return text

In [14]:
df['text'] = df['text'].apply(clear_text)
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww! he matches this background colour i'm s...,0
2,"hey man, i'm really not trying to edit war. it...",0
3,more i can't make any real suggestions on impr...,0
4,"you, sir, are my hero. any chance you remember...",0


Создадим корпус, текст приведем к формату Unicode

In [15]:
corpus = df['text'].values.astype('U')

**Токенизация** 

In [16]:
def tokenize(words):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(words)
    return tokens

In [ ]:
df['tokens'] = df['text'].apply(tokenize)

**Удаление стоп слов**

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

In [ ]:
def remove_stopwords(tokens):
    filtered = [w for w in tokens if not w in stopwords]
    return filtered

In [ ]:
df['no_stop'] = df['tokens'].apply(remove_stopwords)

**Лемматизация**

In [ ]:
def lemmatize(tokens):
    wnl = WordNetLemmatizer()
    lemm = [wnl.lemmatize(word) for word in tokens]
    return lemm

In [ ]:
df['lemm'] = df['no_stop'].apply(lemmatize)

In [ ]:
df.head()

**Вывод**:
- Исследовали и обработали данные.
- Преобразовали датасет и получили леммы слов. 

## Обучение

In [ ]:
features = df['lemm']
target = df['toxic']

делим данные на две выборки трейн и тест  70 к 30

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.25, random_state=42)

 создадим счётчик

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [ ]:
corpus_train = features_train.astype('U')
corpus_test = features_test.astype('U')

посчитаем TF-IDF для корпуса 

In [ ]:
tf_idf = count_tf_idf.fit_transform(corpus_train)

LogisticRegression

In [ ]:
%%time
lr = LogisticRegression(random_state=42)
parameters = {'penalty':['l1', 'l2'],
              'C': [10,100],
              'solver':['liblinear'],
              'max_iter':[100,200],
              'class_weight':['balanced']}

grid = GridSearchCV(lr, param_grid = parameters, scoring = 'f1', cv = 3)

train_grid = grid.fit(tf_idf,target_train)
predictions_train = grid.predict(tf_idf)

f1_lr_train = f1_score(target_train, predictions_train)
print('f1 на тренировочной выборке:', f1_lr_train)

In [ ]:
grid.best_params_ 

LGBMClassifier

In [ ]:
%%time
lgbm = LGBMClassifier(random_state=42, class_weight='balanced')

lgbm.fit(tf_idf,target_train)
predictions_train = lgbm.predict(tf_idf)

f1_lgbm_train = f1_score(target_train, predictions_train)
print('f1 на тренировочной выборке:', f1_lgbm_train)

In [ ]:
lr_pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

parameters = {'clf__penalty':['l1', 'l2'],
          'clf__C': [10,100],
          'clf__solver':['liblinear'],
          'clf__max_iter':[100,200],
          'clf__class_weight': ['balanced']}

grid = GridSearchCV(estimator=lr_pipe, param_grid=parameters, cv=3, scoring='f1')
grid.fit(features_train, target_train)
lr_best_parameters = grid.best_params_

print(lr_best_parameters)
print(grid.best_score_)

### Тестирование

In [ ]:
tf_idf_test = count_tf_idf.transform(corpus_test)

LogisticRegression

In [ ]:
%%time
lr = LogisticRegression(random_state=42, C = 10, class_weight = 'balanced', 
                        max_iter = 100, penalty = 'l1', solver = 'liblinear')
lr.fit(tf_idf,target_train)
predictions_test = lr.predict(tf_idf_test)
f1_lr_test = f1_score(target_test, predictions_test)

print('f1 логистической регрессии на тестовой выборке:', f1_lr_test)

LGBM

In [ ]:
%%time
predictions_test = lgbm.predict(tf_idf_test)
f1_lgbm_test = f1_score(target_test, predictions_test)

print('f1 LightGBM на тестовой выборке:', f1_lgbm_test)

## Выводы

В ходе данного проекта были обучены модели LogisticRegression и LGBMClassifier, чтобы классифицировать комментарии на позитивные и негативные. Перед этим комментарии были токенизированы, очищены от стоп-слов и лемматизированы.

Лучший результат f1 метрики показала модель логистической регрессии - 0.9142 на тренировочной выборке и 0.7576 на тестовой.

In [ ]:
summary = pd.DataFrame(columns = ["LogisticRegression","LGBMClassifier"],
                      index = ["f1_train",
                               "f1_test"])


summary.iloc[0] = [f1_lr_train,f1_lgbm_train]
summary.iloc[1] = [f1_lr_test,f1_lgbm_test]

In [ ]:
summary

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны